# Gaps Table
First, make a simple **gaps** table, then fill it with data from other tables.

In [ ]:
%%bigquery udf -m orders_create_additional_fields
/**
 * Pad with 0 or given string.
 *
 * @param int n Number to add padding to.
 * @param int width Width of number + padding.
 * @param string z (Optional) Other string to replace '0' as padding.
 */
function pad(n, width, z) {
  z = z || '0';
  n = n + '';
  return n.length >= width ? n : new Array(width - n.length + 1).join(z) + n;
}

/**
 * Create additional fields on orders table for gaps table creation.
 *
 * @param {{order_id: string, driver_id: string, passenger_id: string,
            start_district_hash: string, dest_district_hash: string, price: float,
            time: string}} r
 * @param function({{order_id: string, driver_id: string, passenger_id: string,
                     start_district_hash: string, dest_district_hash: string, price: float,
                     time: string, timeslot: string, timeofday_slot: integer, day_in_week: integer,
                     date: string}}) emitFn
 */
function(r, emitFn) {
  var t = r.time.split(/[ :\-]/);
  var slot = Math.floor((parseInt(t[3]) * 60 + parseInt(t[4])) / 10) + 1;
  r.timeslot = t[0] + '-' + pad(t[1], 2) +
               '-' + pad(t[2], 2) + '-' + slot;
  r.timeofday_slot = slot;
  r.date = t[0] + '-' + pad(t[1], 2) + '-' + pad(t[2], 2);
  r.day_in_week = new Date(parseInt(t[0]), parseInt(t[1])-1, parseInt(t[2])).getDay();
  emitFn(r);
}

In [ ]:
%%bigquery execute -t datalab-projects-1331:xjk_algo_comp.gaps -m create
SELECT district_id, FIRST(orders.timeslot) AS timeslot, FIRST(orders.date) AS date,
  FIRST(day_in_week) AS day_in_week, FIRST(timeofday_slot) AS timeofday_slot,
  SUM(price) AS sum_price, AVG(price) AS avg_price,
  SUM(IF(driver_id = 'NULL', 1, 0)) AS gap
FROM orders_create_additional_fields([datalab-projects-1331:xjk_algo_comp.orders]) AS orders
JOIN [datalab-projects-1331:xjk_algo_comp.districts] AS districts 
  ON orders.start_district_hash = districts.district_hash
GROUP BY district_id, orders.timeslot

In [ ]:
%% sql -q tester
SELECT SUM(price) FROM datalab-projects-1331:xjk_algo_comp.orders
WHERE start_district_hash = 'd4ec2125aff74eded207d2d915ef682f'
  AND (time LIKE "%2016-01-01 00:5%")
  
# Reference only. Use above query to check if aggregation was correct, by comparing the
# result of that from sum of district_hash = 51 and timeslot "2016-01-01-6", which 
# should be 126498.0.

# +1 timeslot
We are going to use past data to predict the gaps for the next 10 minutes (or in other words, the next timeslot). To do this, we need to update timeslot related information. Specifically we are going to do the following:
1. Add `timeslot` feature by one:
  - '2016-01-45' to '2016-01-46'
  - '2016-01-144' to '2016-02-1'
2. Update `date` and `slot` features accordingly.
3. Recalculate `day_in_week` feature.

In [ ]:
%%bigquery udf --module gaps_plus_one_timeslot
/**
 * Pad with 0 or given string.
 *
 * @param int n Number to add padding to.
 * @param int width Width of number + padding.
 * @param string z (Optional) Other string to replace '0' as padding.
 */
function pad(n, width, z) {
  z = z || '0';
  n = n + '';
  return n.length >= width ? n : new Array(width - n.length + 1).join(z) + n;
}

/**
 * Add one timeslot and adjust other relevant tables.
 *
 * @param {{district_id: integer, timeslot: string, date: string, day_in_week: integer,
            timeofday_slot: integer, sum_price: float, avg_price: float, gap: integer}} r
 * @param function({{district_id: integer, timeslot: string, date: string, day_in_week: integer,
                timeofday_slot: integer, sum_price: float, avg_price: float, gap: integer}}) emitFn
 */
function(r, emitFn) {
  
  var t = r.timeslot.split(/-/);
  var oldslot = parseInt(t[3]);
  var newslot = oldslot + 1;
  var d = new Date(parseInt(t[0]), parseInt(t[1])-1, parseInt(t[2]));
  if (oldslot == 144) {
    newslot = 1;
    d = d.setDate(d.getDate() + 1);
    r.day_in_week = d.getDay();
    r.date = d.getFullYear() + '-' + pad(d.getMonth()+1, 2) +
             '-' + pad(d.getDate(), 2);
  }
  r.timeslot = d.getFullYear() + '-' + pad(d.getMonth()+1, 2) +
               '-' + pad(d.getDate(), 2) + '-' + newslot;

  r.timeofday_slot = newslot;
  emitFn(r);
}

In [ ]:
%%bigquery udf --module gaps_plus_one_timeslot
  
// Use this when testing
// %%javascript

/**
 * Pad with 0 or given string.
 *
 * @param int n Number to add padding to.
 * @param int width Width of number + padding.
 * @param string z (Optional) Other string to replace '0' as padding.
 */
function pad(n, width, z) {
  z = z || '0';
  n = n + '';
  return n.length >= width ? n : new Array(width - n.length + 1).join(z) + n;
}

/**
 * Add one timeslot and adjust other relevant tables.
 *
 * @param {{district_id: integer, timeslot: string, date: string, day_in_week: integer,
            timeofday_slot: integer, sum_price: float, avg_price: float, gap: integer}} r
 * @param function({{district_id: integer, timeslot: string, date: string, day_in_week: integer,
                timeofday_slot: integer, sum_price: float, avg_price: float, gap: integer}}) emitFn
 */
function(r, emitFn) {
  
  var t = r.timeslot.split(/-/);
  var oldslot = parseInt(t[3]);
  var newslot = oldslot + 1;
  var d = new Date(parseInt(t[0]), parseInt(t[1])-1, parseInt(t[2]));
  if (oldslot == 144) {
    newslot = 1;
    d = new Date(d.setDate(d.getDate() + 1));
    r.day_in_week = d.getDay();
    r.date = d.getFullYear() + '-' + pad(d.getMonth()+1, 2) +
             '-' + pad(d.getDate(), 2);
  }
  r.timeslot = d.getFullYear() + '-' + pad(d.getMonth()+1, 2) +
               '-' + pad(d.getDate(), 2) + '-' + newslot;

  r.timeofday_slot = newslot;
  emitFn(r);
}

// For testing (without `new Date()` after date added by 1 setDate returns milliseconds)

// var test_row = {
//   district_id: 1,
//   timeslot: '2016-01-22-144',
//   date: '2016-01-22',
//   day_in_week: 5,
//   timeofday_slot: 144,
//   sum_price: 0.0,
//   avg_price: 0.0,
//   gap: 11
// };

// function emitter(r) {
//   for (var p in r) {
//     element.append(p + '=' + r[p] + '<br>');
//   }
// }

// udf(test_row, emitter);

In [ ]:
%%bigquery execute -t datalab-projects-1331:xjk_algo_comp.future_gaps -m create

SELECT district_id, timeslot, date, day_in_week, timeofday_slot, sum_price, avg_price, gap
FROM gaps_plus_one_timeslot([datalab-projects-1331:xjk_algo_comp.gaps])

# Gaps Table + Other Tables

In [ ]:
%%bigquery execute -t datalab-projects-1331:xjk_algo_comp.future_gaps_processed -m create

SELECT FIRST(gaps.district_id) AS district_id, FIRST(gaps.timeslot) AS timeslot, FIRST(gaps.date) AS date,
  FIRST(gaps.day_in_week) AS day_in_week, IF(FIRST(gaps.day_in_week) = 0, 1, 0) AS is_sunday,
  FIRST(gaps.timeofday_slot) AS timeofday_slot,
  FIRST(gaps.sum_price) AS sum_price, FIRST(gaps.avg_price) AS avg_price, FIRST(gaps.gap) AS gap,
  FIRST(pois.f1) AS f1, FIRST(pois.f11) AS f11, FIRST(pois.f11_1) AS f11_1, 
  FIRST(pois.f11_2) AS f11_2, FIRST(pois.f11_3) AS f11_3, FIRST(pois.f11_4) AS f11_4, 
  FIRST(pois.f11_5) AS f11_5, FIRST(pois.f11_6) AS f11_6, FIRST(pois.f11_7) AS f11_7, 
  FIRST(pois.f11_8) AS f11_8, FIRST(pois.f13_4) AS f13_4, FIRST(pois.f13_8) AS f13_8, 
  FIRST(pois.f14) AS f14, FIRST(pois.f14_1) AS f14_1, FIRST(pois.f14_10) AS f14_10, 
  FIRST(pois.f14_2) AS f14_2, FIRST(pois.f14_3) AS f14_3, FIRST(pois.f14_6) AS f14_6, 
  FIRST(pois.f14_8) AS f14_8, FIRST(pois.f15) AS f15, FIRST(pois.f15_1) AS f15_1, 
  FIRST(pois.f15_2) AS f15_2, FIRST(pois.f15_3) AS f15_3, FIRST(pois.f15_4) AS f15_4, 
  FIRST(pois.f15_6) AS f15_6, FIRST(pois.f15_7) AS f15_7, FIRST(pois.f15_8) AS f15_8, 
  FIRST(pois.f16) AS f16, FIRST(pois.f16_1) AS f16_1, FIRST(pois.f16_10) AS f16_10, 
  FIRST(pois.f16_11) AS f16_11, FIRST(pois.f16_12) AS f16_12, FIRST(pois.f16_3) AS f16_3, 
  FIRST(pois.f16_4) AS f16_4, FIRST(pois.f16_6) AS f16_6, FIRST(pois.f17) AS f17, 
  FIRST(pois.f17_2) AS f17_2, FIRST(pois.f17_3) AS f17_3, FIRST(pois.f17_4) AS f17_4, 
  FIRST(pois.f17_5) AS f17_5, FIRST(pois.f19) AS f19, FIRST(pois.f19_1) AS f19_1, 
  FIRST(pois.f19_2) AS f19_2, FIRST(pois.f19_3) AS f19_3, FIRST(pois.f19_4) AS f19_4, 
  FIRST(pois.f1_1) AS f1_1, FIRST(pois.f1_10) AS f1_10, FIRST(pois.f1_11) AS f1_11, 
  FIRST(pois.f1_2) AS f1_2, FIRST(pois.f1_3) AS f1_3, FIRST(pois.f1_4) AS f1_4, 
  FIRST(pois.f1_5) AS f1_5, FIRST(pois.f1_6) AS f1_6, FIRST(pois.f1_7) AS f1_7, 
  FIRST(pois.f1_8) AS f1_8, FIRST(pois.f20) AS f20, FIRST(pois.f20_1) AS f20_1, 
  FIRST(pois.f20_2) AS f20_2, FIRST(pois.f20_4) AS f20_4, FIRST(pois.f20_5) AS f20_5, 
  FIRST(pois.f20_6) AS f20_6, FIRST(pois.f20_7) AS f20_7, FIRST(pois.f20_8) AS f20_8, 
  FIRST(pois.f20_9) AS f20_9, FIRST(pois.f21_1) AS f21_1, FIRST(pois.f21_2) AS f21_2, 
  FIRST(pois.f22) AS f22, FIRST(pois.f22_1) AS f22_1, FIRST(pois.f22_2) AS f22_2, 
  FIRST(pois.f22_3) AS f22_3, FIRST(pois.f22_4) AS f22_4, FIRST(pois.f22_5) AS f22_5, 
  FIRST(pois.f23) AS f23, FIRST(pois.f23_1) AS f23_1, FIRST(pois.f23_2) AS f23_2, 
  FIRST(pois.f23_3) AS f23_3, FIRST(pois.f23_4) AS f23_4, FIRST(pois.f23_5) AS f23_5, 
  FIRST(pois.f23_6) AS f23_6, FIRST(pois.f24) AS f24, FIRST(pois.f24_1) AS f24_1, 
  FIRST(pois.f24_2) AS f24_2, FIRST(pois.f24_3) AS f24_3, FIRST(pois.f25) AS f25, 
  FIRST(pois.f25_1) AS f25_1, FIRST(pois.f25_3) AS f25_3, FIRST(pois.f25_7) AS f25_7, 
  FIRST(pois.f25_8) AS f25_8, FIRST(pois.f25_9) AS f25_9, FIRST(pois.f2_1) AS f2_1, 
  FIRST(pois.f2_10) AS f2_10, FIRST(pois.f2_11) AS f2_11, FIRST(pois.f2_12) AS f2_12, 
  FIRST(pois.f2_13) AS f2_13, FIRST(pois.f2_2) AS f2_2, FIRST(pois.f2_4) AS f2_4, 
  FIRST(pois.f2_5) AS f2_5, FIRST(pois.f2_6) AS f2_6, FIRST(pois.f2_7) AS f2_7, 
  FIRST(pois.f2_8) AS f2_8, FIRST(pois.f3_1) AS f3_1, FIRST(pois.f3_2) AS f3_2, 
  FIRST(pois.f3_3) AS f3_3, FIRST(pois.f4) AS f4, FIRST(pois.f4_1) AS f4_1, 
  FIRST(pois.f4_10) AS f4_10, FIRST(pois.f4_11) AS f4_11, FIRST(pois.f4_13) AS f4_13, 
  FIRST(pois.f4_14) AS f4_14, FIRST(pois.f4_16) AS f4_16, FIRST(pois.f4_17) AS f4_17, 
  FIRST(pois.f4_18) AS f4_18, FIRST(pois.f4_2) AS f4_2, FIRST(pois.f4_3) AS f4_3, 
  FIRST(pois.f4_5) AS f4_5, FIRST(pois.f4_6) AS f4_6, FIRST(pois.f4_7) AS f4_7, 
  FIRST(pois.f4_8) AS f4_8, FIRST(pois.f4_9) AS f4_9, FIRST(pois.f5) AS f5, 
  FIRST(pois.f5_1) AS f5_1, FIRST(pois.f5_3) AS f5_3, FIRST(pois.f5_4) AS f5_4, 
  FIRST(pois.f6) AS f6, FIRST(pois.f6_1) AS f6_1, FIRST(pois.f6_2) AS f6_2, 
  FIRST(pois.f6_4) AS f6_4, FIRST(pois.f7) AS f7, FIRST(pois.f8) AS f8, 
  FIRST(pois.f8_1) AS f8_1, FIRST(pois.f8_2) AS f8_2, FIRST(pois.f8_3) AS f8_3, 
  FIRST(pois.f8_4) AS f8_4, FIRST(pois.f8_5) AS f8_5,
  FIRST(weather.weather) AS weather,
  FIRST(weather.temperature) AS weather_temperature, FIRST(weather.pm25) AS weather_pm25,
  FIRST(traffic.tj_level1) AS traffic_tj_level1, FIRST(traffic.tj_level2) AS traffic_tj_level2,
  FIRST(traffic.tj_level3) AS traffic_tj_level3, FIRST(traffic.tj_level4) AS traffic_tj_level4
FROM [datalab-projects-1331:xjk_algo_comp.future_gaps] as gaps
LEFT JOIN [datalab-projects-1331:xjk_algo_comp.districts] as districts
  ON districts.district_id = gaps.district_id
LEFT JOIN [datalab-projects-1331:xjk_algo_comp.pois] as pois
  ON pois.district_hash = districts.district_hash
LEFT JOIN [datalab-projects-1331:xjk_algo_comp.weather] as weather
  ON weather.timeslot = gaps.timeslot
LEFT JOIN [datalab-projects-1331:xjk_algo_comp.traffic] as traffic
  ON traffic.timeslot = gaps.timeslot
  AND traffic.district_hash = districts.district_hash
GROUP BY gaps.district_id, gaps.timeslot

# PCA
Combine POIs (features starting with 'f...') into n number of features.

In [ ]:
%%sql --module query
SELECT * FROM [datalab-projects-1331:xjk_algo_comp.future_gaps_processed]

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import gcp.bigquery as bq

df = bq.Query(query).to_dataframe()

In [ ]:
pca = PCA(n_components=3)
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["srch_destination_id"] = destinations["srch_destination_id"]